<a href="https://colab.research.google.com/github/Titoffifee/AltGU/blob/main/AGU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install pymorphy2

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [47]:
import pandas as pd
import numpy as np

workers = pd.read_csv('/content/drive/MyDrive/Владивосток/AltGU/employees.csv')
train = pd.read_csv('/content/drive/MyDrive/Владивосток/train_dataset_train/train_issues.csv')
test = pd.read_csv('/content/drive/MyDrive/Владивосток/test_dataset_test/test_issues.csv')

In [48]:
train = train.drop(['id', 'created', 'key', 'project_id'], axis=1)
test = test.drop(['id', 'created', 'key', 'project_id'], axis=1)

In [49]:
ids = workers['id']
workers['position'] = workers['position'].fillna('Web-разработчик')
workers['payment_type'] = workers['payment_type'].fillna('fixed')
test['summary'] = test['summary'].fillna('NAN')
train['summary'] = train['summary'].fillna('NAN')
d = workers[['active', 'position', 'payment_type']].to_dict()

In [ ]:
import pymorphy2
import re

patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\]+-0123456789"
table = dict()
for el in patterns:
  table[ord(el)] = ' '
stopwords_ru = nltk.corpus.stopwords.words("russian")
stopwords_en = nltk.corpus.stopwords.words("english")
stop = stopwords_en + stopwords_ru

morph = pymorphy2.MorphAnalyzer()


def lemmatize(doc):
  doc = doc.translate(table)
  print(doc)
  tokens = []
  for token in doc.split():
    if token and token not in stop:
      token = token.strip()
      token = morph.normal_forms(token)[0]
      
      tokens.append(token)
  return ' '.join(tokens)

test['summary'] = test['summary'].apply(lemmatize)
train['summary'] = train['summary'].apply(lemmatize)

In [71]:
words_cnt = dict()
for text in pd.concat([train['summary'], test['summary']]):
  if type(text) == float:
    continue
  for token in text.split():
    if token not in words_cnt:
      words_cnt[token] = 0
    words_cnt[token] += 1
words_cnt = sorted(words_cnt.items(), key=lambda p: p[1])
while words_cnt[0][1] == 1:
  del words_cnt[0]
words_cnt = set(map(lambda x: x[0], words_cnt))

In [64]:
def GetActive(ind):
  if ind not in ids:
    return 0 
  return d['active'][ind]

def GetPosition(ind):
  if ind not in ids:
    return 'Web-разработчик'
  return d['position'][ind]

def GetPayment(ind):
  if ind not in ids:
    return 'fixed'
  return d['payment_type'][ind]

def GetCorrectData(series):
  active = series.apply(GetActive).rename('employee_active')
  position = series.apply(GetPosition).rename('employee_position')
  payment_type = series.apply(GetPayment).rename('employee_payment_type')
  return pd.concat([active, position, payment_type], axis=1)

X, y = train.drop(['overall_worklogs'], axis=1), train['overall_worklogs']

X = pd.concat([X, GetCorrectData(X['assignee_id'])], axis=1)
X = X.drop('creator_id', axis=1)

In [72]:
X_test = pd.concat([test, GetCorrectData(test['assignee_id'])], axis=1)
X_test = X_test.drop(['creator_id'], axis=1)

In [74]:
summary_X = pd.DataFrame(index=range(X.shape[0]), columns=list(words_cnt))
summary_X_test = pd.DataFrame(index=range(X_test.shape[0]), columns=list(words_cnt))

In [75]:
ind = 0
for text in X['summary']:
  if type(text) == float:
    continue
  for token in text.split():
    if token in words_cnt:
      summary_X.at[ind, token] = 1
  ind += 1

ind = 0
for text in X_test['summary']:
  if type(text) == float:
    continue
  for token in text.split():
    if token in words_cnt:
      summary_X_test.at[ind, token] = 1
  ind += 1

In [76]:
summary_X = summary_X.fillna(0)
summary_X_test = summary_X_test.fillna(0)

In [77]:
X = pd.concat([X.drop(['summary'], axis=1), summary_X], axis=1)
X_test = pd.concat([X_test.drop(['summary'], axis=1), summary_X_test], axis=1)

In [82]:
y /= 60

In [ ]:
obj = ['employee_position', 'employee_payment_type', 'assignee_id']
X = X.drop(obj + ['employee_active'], axis=1)
X_test = X_test.drop(obj + ['employee_active'], axis=1)

In [124]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(silent=True, random_seed=0,
                          depth=8, iterations=3200, learning_rate=0.04)
# 6 100 0.01 - -0.01616

In [125]:
from sklearn.model_selection import cross_validate

result = cross_validate(model, X, y, cv=5, scoring='r2')

In [126]:
print(result['test_score'])
print(result['test_score'].mean())

[-6.95623732  0.30586933 -0.04512779 -3.56939026 -0.06207206]
-2.065391621098176


In [120]:
model.fit(X, y)
result = model.predict(X_test)
result *= 60
result

array([16214.88230873, 16214.88230873, 16266.34257253, ...,
       16245.8644799 , 16214.88230873, 16174.93262614])

In [97]:
result[result <= 500]

array([], dtype=float64)

In [ ]:
df_result = pd.read_csv('/content/drive/MyDrive/Владивосток/AltGU/sample_solution.csv')
df_result['overall_worklogs'] = result
df_result.to_csv('/content/drive/MyDrive/Владивосток/ans.csv', index=False)
df_result